In [2]:
import os
import glob
import numpy as np
import pandas as pd
import statsmodels.api as sm

from pydicom import dcmread
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
homedir = '/home/raghuram/Desktop/radiomics/TEXTURES/csv_folder'
os.chdir(homedir)

In [4]:
random_seed=5

In [10]:
t1_list = glob.glob('*_T1W_features.csv')
t2_list = glob.glob('*_T2W_features.csv')
t1ce_list = glob.glob('*_T1CE_features.csv')
flair_list = glob.glob('*_T2F_features.csv')

In [ ]:
def pre_process_dataframe(sequence_dataframe_raw, sequence_list):
    # Binarize magnetization strength values
    # Scanner names and manufacturer are categorical 
    # One hot encode them
    
    # Code for the above here
    # Experiments in ascending order
    for
    for files in sequence_list:
        
    
    
    pass

In [ ]:
def split_data_train_test():
    # Split the preprocessed data into train and test
    
    pass

In [ ]:
def linear_regression(sequence, target_variable):
    # Load the data corresponding to the sequence argument here
    # Perform regression with Tr and Te 
    pass